In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os

In [ ]:
data = pd.read_pickle('Output/test.pkl')

models = [
    'fold_0_iter',
    'fold_2_iter',
    'fold_4_iter',
    'fold_1_0_iter',
    'fold_1_3_iter',
    'fold_1_4_iter',
    'fold_3_0_iter',
    'fold_3_1_iter',
    'fold_3_3_iter',
    'weak_fold_'
]

model_list = []
for path in ['Models_lgbm_DART/', 'Models/']:
    for fname in os.listdir(path):
        for model_name in models:
            if model_name in fname:
                model_list.append(path + fname)

pred_list = []
for model_path in model_list:
    print(model_path)
    print('{:.02f} MB'.format(os.path.getsize(model_path)/1000000))
    model = joblib.load(model_path)
    pred_list.append(model.predict(data))

column_name = [
    'fold_3_1',
    'fold_2',
    'fold_0',
    'fold_3_0',
    'fold_3_3',
    'fold_1_3',
    'fold_1_4',
    'fold_4',
    'fold_1_0',
    'weak_fold_7',
    'weak_fold_1',
    'weak_fold_5',
    'weak_fold_9',
    'weak_fold_2',
    'weak_fold_3',
    'weak_fold_4',
    'weak_fold_0',
    'weak_fold_8',
    'weak_fold_6',
]

pred_df = pd.DataFrame(pred_list).T
pred_df.columns = column_name
pred_df.index = data.index

In [ ]:
pred_df.to_csv('Output/DART_folds.csv')
pred_df.mean(axis=1).to_csv('Output/p_DART_fold.csv', header=['prediction'])

In [4]:
pred_df = pd.read_csv('Output/DART_folds.csv', index_col='customer_ID')

In [9]:
study_df = pd.DataFrame(pred_df.mean(axis=1), columns=['mean'])
study_df['std'] = pred_df.std(axis=1)
study_df


,mean,std
customer_ID,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.025017,0.005987
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.000774,0.000269
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.044970,0.010046
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.238053,0.026484
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.866483,0.029297
...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.010842,0.001401
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.779906,0.024950
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.378065,0.073314
